# Importing libraries

Loading all of the dependencies.

In [ ]:
import pandas as pd
from sklearn import datasets
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Perceptron Implementation

Implementing Multilayer Perceptron with functionality to choose activation function, amount and size of layers.

In [ ]:
import numpy as np


class ActivationFunctions:

    @staticmethod
    def sigmoid(x):
        return 1.0 / (1.0 + np.exp(-x))

    @staticmethod
    def sigmoid_derivative(x):
        return x * (1 - x)

    @staticmethod
    def softmax(x):
        y = np.exp(x - x.max())
        return y / y.sum()


class MLP:
    def __init__(self, inputs_count=2, hidden_layers_counts_with_act_funcs=[
        (2, ActivationFunctions.sigmoid, ActivationFunctions.sigmoid_derivative)],
                 outputs_count_with_act_funcs=[
                     (1, ActivationFunctions.sigmoid, ActivationFunctions.sigmoid_derivative)]):
        self.inputs_count = inputs_count
        self.hidden_layers_counts_with_act_funcs = hidden_layers_counts_with_act_funcs
        self.outputs_counts_with_act_funcs = outputs_count_with_act_funcs

        self.layers_counts_with_funcs = [(self.inputs_count, None, None)] + self.hidden_layers_counts_with_act_funcs \
                                        + self.outputs_counts_with_act_funcs

        self.weights = []
        for i in range(len(self.layers_counts_with_funcs) - 1):
            self.weights.append((np.random.rand(self.layers_counts_with_funcs[i][0],
                                               self.layers_counts_with_funcs[i + 1][0])) -
                                np.full(self.layers_counts_with_funcs[i][0], self.layers_counts_with_funcs[i + 1][0], 0.5))

        self.derivatives = []
        for i in range(len(self.layers_counts_with_funcs) - 1):
            self.derivatives.append(np.zeros((self.layers_counts_with_funcs[i][0],
                                              self.layers_counts_with_funcs[i + 1][0])))

        self.act_parameters = []
        for i in range(len(self.layers_counts_with_funcs)):
            self.act_parameters.append(np.zeros(self.layers_counts_with_funcs[i][0]))

    def predict(self, inputs):
        currents_layers_act_parameters = inputs
        self.act_parameters[0] = inputs

        for index, matrix in enumerate(self.weights):
            current_layer_inputs = np.dot(currents_layers_act_parameters, matrix)
            currents_layers_act_parameters = self.layers_counts_with_funcs[index + 1][1](current_layer_inputs)
            self.act_parameters[index + 1] = currents_layers_act_parameters

        return currents_layers_act_parameters

    def fit(self, X, y, epochs, learning_rate):
        for i in range(epochs):
            for j, x in enumerate(X):
                y_obj = y[j]
                output = self.predict(x)
                error = y_obj - output

                for k in reversed(range(len(self.derivatives))):
                    delta = error * self.layers_counts_with_funcs[k + 1][2](self.act_parameters[k + 1])
                    delta_transposed = delta.reshape(delta.shape[0], -1).T
                    current_act_parameters = self.act_parameters[k]
                    current_act_parameters = current_act_parameters.reshape(current_act_parameters.shape[0], -1)
                    self.derivatives[k] = np.dot(current_act_parameters, delta_transposed)
                    error = np.dot(delta, self.weights[k].T)

                for k in range(len(self.weights)):
                    matrix = self.weights[k]
                    derivatives = self.derivatives[k]
                    matrix += derivatives * learning_rate

        return self


# Loading Titanic Dataset

### Loading data

In [ ]:
data = pd.read_csv('../data/train.csv', sep = ',')

In [ ]:
data.head()

### Preprocessing the data

Removing some attributes as they are not so important for our model and also filling empty spaces.

In [ ]:
data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
data.fillna(data.mean(), inplace=True)
data

### One-Hot Encoding

In [ ]:
data = pd.get_dummies(data=data, prefix='c', columns=['Sex', 'Embarked'])
data

### Separating the data

In [ ]:
y = data[['Survived']]
X = data[data.columns[1:data.shape[1]]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

### Building a model

In [ ]:
model = MLP(inputs_count=X_train.shape[1], hidden_layers_counts_with_act_funcs=[(15, ActivationFunctions.sigmoid, ActivationFunctions.sigmoid_derivative),
                                                                      (15, ActivationFunctions.sigmoid, ActivationFunctions.sigmoid_derivative)])\
    .fit(X_train.values, y_train.values, epochs=600, learning_rate=0.25)

### Fitting and estimating the model

In [ ]:
y_pred = []
for i in range(len(X_test.values)):
    prediction = model.predict(X_test.values[i])[0]
    if prediction > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)
        
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Loading Iris Dataset

### Loading data

In [ ]:
iris = datasets.load_iris()

### Splitting data

In [ ]:
X = iris.data
y = iris.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

### Scaling

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Building a model

In [ ]:
model = MLP(inputs_count=X_train.shape[1], hidden_layers_counts_with_act_funcs=[(3, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative),
                                                                      (3, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative)],
                outputs_count_with_act_funcs=[(3, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative)])\
    .fit(X_train, y_train, epochs=300, learning_rate=0.15)

In [ ]:
y_pred = []
for i in range(len(X_test)):
    y_pred.append(np.argmax(model.predict(X_test[i])))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))